# Overview:

This notebook is used to prepare the data for the index. The data is prepared in the following steps:
1. Read in the data
2. Merge the macro data with the index price data
3. Merge the fundamental data with the index price data
4. Create technical features using the index price data
5. Save the data as a csv file

In [ ]:
import pandas as pd

In [2]:
# read in macro and  index price / fundamental data
index_prices = pd.read_csv("data/index_prices.csv")
macro_data = pd.read_csv("data/macro_data.csv")
fundamentals = pd.read_excel("data/SPXEWCS DATA VALUES.xlsx")
index_prices

,Date,Adj Close
0,2006-11-07,1.743284
1,2006-11-08,1.743284
2,2006-11-09,1.743284
3,2006-11-10,1.747295
4,2006-11-13,1.747295
...,...,...
4272,2023-10-30,28.910000
4273,2023-10-31,29.049999
4274,2023-11-01,28.930000
4275,2023-11-02,29.430000


In [3]:
# dictionary explaining the macro data
data_dict = {
    'NA000334Q': 'Gross Domestic Product, Quarterly, Not Seasonally Adjusted',
    'REAINTRATREARAT10Y': '10-Year Real Interest Rate',
    'UNRATE': 'Unemployment Rate, Quarterly, Not Seasonally Adjusted',
    'CPALTT01USQ661S': 'Consumer Price Index: All Items: Total for United States, Quarterly, Seasonally Adjusted',
    'PPIACO': 'Producer Price Index by Commodity: All Commodities, Monthly, Not Seasonally Adjusted',
    'HOUST': 'New Privately-Owned Housing Units Started: Total Units',
    'PSAVERT': 'Personal Saving Rate, Monthly, Seasonally Adjusted',
}

# rename columns to be more descriptive
macro_data = macro_data.rename(columns={
    'NA000334Q': 'GDP',
    'REAINTRATREARAT10Y': 'RATE_10',
    'UNRATE': 'URATE',
    'CPALTT01USQ661S': 'CPI',
    'PPIACO': 'PPI',
    'HOUST': 'HOUSE',
    'PSAVERT': 'SAVE',
})

In [4]:
# subset macro data to only have what's needed (data without nans and only for when index is trading)
macro_data = macro_data[macro_data['DATE'] > '2006-10'] # select only data for when index is trading
macro_data = macro_data.iloc[:-1] # remove last row because of NAs

In [5]:
# forward fill data for GDP and CPI columns
# since these are quarterly data, we can forward fill the data so that the last reported value is used for the entire quarter
macro_data = macro_data.fillna(method='ffill')

In [6]:
# convert date columns to datetime so that we can merge the dataframes
macro_data['DATE'] = pd.to_datetime(macro_data['DATE'])
index_prices['Date'] = pd.to_datetime(index_prices['Date'])

In [7]:
# create dictionary of index data from merging index_prices and macro_data
#
index_data_dict = {}

# for each time step in index_prices, find the corresponding macro data merge it with the price data
# add the resulting dataset to the dictionary
for row in index_prices.iterrows():

    # get index price and date data
    index_date = row[1]['Date']
    index_price = row[1]['Adj Close']
    index_month = row[1]['Date'].month
    index_year = row[1]['Date'].year

    # append macro data to index_prices with matching month and year
    macro_row = macro_data[(macro_data['DATE'].dt.month == index_month) & (macro_data['DATE'].dt.year == index_year)]

    # check if macro data exists for the given date
    # if not, print a message and continue to the next date
    if macro_row.empty:
        print(f'No macro data for {index_date}')
        continue
    else:
        # if there is macro data, add it to the dictionary
        # add to dictionary
        index_data_dict[index_date] = [index_price, macro_row['GDP'].values[0], macro_row['RATE_10'].values[0], macro_row['URATE'].values[0], macro_row['CPI'].values[0], macro_row['PPI'].values[0], macro_row['HOUSE'].values[0], macro_row['SAVE'].values[0]]


No macro data for 2023-11-01 00:00:00
No macro data for 2023-11-02 00:00:00
No macro data for 2023-11-03 00:00:00


In [8]:
# convert dictionary to dataframe
# the index_data dataframe now has the index price and macro data for each time step (day)
index_data = pd.DataFrame.from_dict(index_data_dict, orient='index', columns=['Price', 'GDP', 'RATE_10', 'URATE', 'CPI', 'PPI', 'HOUSE', 'SAVE'])
index_data

,Price,GDP,RATE_10,URATE,CPI,PPI,HOUSE,SAVE
2006-11-07,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7
2006-11-08,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7
2006-11-09,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7
2006-11-10,1.747295,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7
2006-11-13,1.747295,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7
...,...,...,...,...,...,...,...,...
2023-10-25,29.120001,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8
2023-10-26,29.020000,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8
2023-10-27,28.559999,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8
2023-10-30,28.910000,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8


In [9]:
# prepare fundamentals data to be merged with index_data
fundamentals = fundamentals.iloc[238:4672] # remove rows at beginning with missing data, and select only data for timesteps matching index_data
fundamentals = fundamentals.drop(columns=['Last Price'])
fundementals = fundamentals.rename(columns={'Unnamed: 0': 'Date'})
fundementals['Date'] = pd.to_datetime(fundementals['Date'])
fundementals = fundementals.set_index('Date')
fundementals

,Index Enterprise Value,Adjusted Price/Earnings ratio,Adjusted Price/Estimated Earnings,BEst Div Yld,Total Debt to Total Equity,Current Market Cap,Profit Margin
Date,,,,,,,
2006-11-02,2123.3586,9.7138,19.9605,1.9262,105.0283,1286569.94,4.0568
2006-11-03,2101.3037,9.5817,19.66,1.9571,105.0283,1277994.84,4.0568
2006-11-06,2115.073,9.6612,19.8239,1.9409,105.0283,1284279.01,4.0568
2006-11-07,2112.1243,9.6442,19.7364,1.9466,105.0283,1284884.54,4.0568
2006-11-08,2122.9425,9.7067,19.8696,1.9357,105.0283,1290784.26,4.0568
...,...,...,...,...,...,...,...
2023-10-25,6645.8868,15.7161,15.997,3.1342,98.9909,2639057.02,3.6098
2023-10-26,6628.8768,15.6644,15.9387,3.1371,98.9909,2622045.95,3.6098
2023-10-27,6550.3068,15.4255,15.685,3.1845,98.9909,2588702.38,3.6098


In [10]:
# check for missing values
fundementals.isna().sum()

Index Enterprise Value               0
Adjusted Price/Earnings ratio        0
Adjusted Price/Estimated Earnings    0
BEst Div Yld                         0
Total Debt to Total Equity           0
Current Market Cap                   0
Profit Margin                        0
dtype: int64

In [11]:
# merge index_data and fundementals
# index_data now has all the data we need for the index (price, macro, and fundamental data)
index_data = index_data.merge(fundementals, how='left', left_index=True, right_index=True)
index_data

,Price,GDP,RATE_10,URATE,CPI,PPI,HOUSE,SAVE,Index Enterprise Value,Adjusted Price/Earnings ratio,Adjusted Price/Estimated Earnings,BEst Div Yld,Total Debt to Total Equity,Current Market Cap,Profit Margin
2006-11-07,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2112.1243,9.6442,19.7364,1.9466,105.0283,1284884.54,4.0568
2006-11-08,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2122.9425,9.7067,19.8696,1.9357,105.0283,1290784.26,4.0568
2006-11-09,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2116.7542,9.6709,19.796,1.9406,105.0283,1277823.39,4.0568
2006-11-10,1.747295,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2118.6522,9.6819,19.8196,1.9375,105.0283,1278345.03,4.0568
2006-11-13,1.747295,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2116.3033,9.6683,19.8379,1.9402,105.0283,1272948.74,4.0568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,29.120001,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6645.8868,15.7161,15.997,3.1342,98.9909,2639057.02,3.6098
2023-10-26,29.020000,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6628.8768,15.6644,15.9387,3.1371,98.9909,2622045.95,3.6098
2023-10-27,28.559999,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6550.3068,15.4255,15.685,3.1845,98.9909,2588702.38,3.6098
2023-10-30,28.910000,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6609.9668,15.6069,15.8603,3.1553,98.9909,2627660.6,3.6098


In [12]:
# use price column to create technical features (moving averages, momentum, etc.)
import talib as ta

# 10, 20, and 60 day moving averages
index_data['SMA_10d'] = ta.SMA(index_data['Price'], timeperiod=10)
index_data['SMA_20d'] = ta.SMA(index_data['Price'], timeperiod=20)
index_data['SMA_60d'] = ta.SMA(index_data['Price'], timeperiod=60)

# exponential moving averages
index_data['EMA_10d'] = ta.EMA(index_data['Price'], timeperiod=10)
index_data['EMA_20d'] = ta.EMA(index_data['Price'], timeperiod=20)
index_data['EMA_60d'] = ta.EMA(index_data['Price'], timeperiod=60)

# momentum
index_data['MOM_10d'] = ta.MOM(index_data['Price'], timeperiod=10)
index_data['MOM_20d'] = ta.MOM(index_data['Price'], timeperiod=20)
index_data['MOM_60d'] = ta.MOM(index_data['Price'], timeperiod=60)

# relative strength index
index_data['RSI_10d'] = ta.RSI(index_data['Price'], timeperiod=10)
index_data['RSI_20d'] = ta.RSI(index_data['Price'], timeperiod=20)
index_data['RSI_60d'] = ta.RSI(index_data['Price'], timeperiod=60)

index_data

,Price,GDP,RATE_10,URATE,CPI,PPI,HOUSE,SAVE,Index Enterprise Value,Adjusted Price/Earnings ratio,...,SMA_60d,EMA_10d,EMA_20d,EMA_60d,MOM_10d,MOM_20d,MOM_60d,RSI_10d,RSI_20d,RSI_60d
2006-11-07,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2112.1243,9.6442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-08,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2122.9425,9.7067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-09,1.743284,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2116.7542,9.6709,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-10,1.747295,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2118.6522,9.6819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-13,1.747295,3557633.0,2.199885,4.5,85.366591,164.600,1570.0,2.7,2116.3033,9.6683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-25,29.120001,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6645.8868,15.7161,...,31.056868,28.991930,29.305068,30.653225,0.170000,-0.809999,-4.557558,46.073209,38.395811,39.246621
2023-10-26,29.020000,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6628.8768,15.6644,...,30.976594,28.997033,29.277919,30.599677,0.620001,-0.980000,-4.816418,43.738261,37.439669,38.901081
2023-10-27,28.559999,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6550.3068,15.4255,...,30.889978,28.917573,29.209546,30.532802,-0.110001,-1.480001,-5.196989,34.739759,33.411001,37.362269
2023-10-30,28.910000,6928858.0,2.082461,3.9,129.118446,255.463,1372.0,3.8,6609.9668,15.6069,...,30.813828,28.916196,29.181018,30.479595,-0.059999,-0.750000,-4.568993,44.408773,38.694427,39.222537


In [14]:
# save index data as csv
index_data.to_csv('data/index_data.csv')